In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from numpy.fft import fft

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (30, 20)

In [13]:
## process_file() takes a string with a path to the .csv
def process_file(pathname):
    res = pd.read_csv(pathname, sep = ',', header = 0, index_col = None, names = ['time', 'seconds', 'z', 'y', 'x'])
    return res


def process_fft_file(pathname):
    res = pd.read_csv(pathname, sep=',', header=0, index_col=None, names=['x', 'y'])
    return res

## butterworth() takes a dataframe with our accelerometer data and applies the Butterworth filter to it. 
## The columns of the DF is overwritten by the filtered values.
## Sources: https://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html 
##               https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html
def butterworth(df):
    nyquist_frequency = 0.5 * 50
    
    low = 0.5 / nyquist_frequency
    high = 2.2 / nyquist_frequency
    
    b, a = signal.butter(3, Wn=[low, high], btype='bandpass')

    df['x'] = signal.filtfilt(b, a, df['x'])
    df['y'] = signal.filtfilt(b, a, df['y'])
    df['z'] = signal.filtfilt(b, a, df['z'])

    return df

## fourier() takes a dataframe with our filtered accelerometer data and produces a new dataframe with x, y pairs.
## This produces the Fourier-transformed data. (I assume)
## Sources: https://stackoverflow.com/questions/66675657/fast-fourier-transform-for-an-accelerometer-in-python
def fourier(df):
    yf = np.abs(fft(df['x'] + df['y'] + df['z']))
    xf = np.linspace(0, 50, len(yf))

    return pd.DataFrame(zip(xf, yf), columns = ['x', 'y'])

## to_row() takes a Fourier-transformed dataframe and transposes its 'y' column into a row.
## This makes it possible to pass the transformed dataframe into an ML model.
def to_row(input, name):
    df = pd.DataFrame(input)
    df = df.T
    df.reset_index(drop = True, inplace = True)
    df.insert(loc = 0, column = 'Name', value = name)

    return df
    


In [14]:

# Processing .csv files, passing them through Butterworth filter, and Fourier transforming them

##### 5-MINUTE DATA
dlp_5min = process_file('processed_data/dlp_5min.csv')
drp_5min = process_file('processed_data/drp_5min.csv')                
dlp_5min_bw = process_file('processed_data/dlp_5min_bw.csv')
drp_5min_bw = process_file('processed_data/drp_5min_bw.csv')
dlp5m_bw_fft = process_fft_file('processed_data/dlp_5min_bw_fft.csv')
drp5m_bw_fft = process_fft_file('processed_data/drp_5min_bw_fft.csv')

mrp_5min = process_file('processed_data/mrp_5min.csv')
mrp_5min_bw = process_file('processed_data/mrp_5min_bw.csv')
mrp5m_bw_fft = process_fft_file('processed_data/mrp_5min_bw_fft.csv')

slp_5min = process_file('processed_data/slp_5min.csv')
srp_5min = process_file('processed_data/srp_5min.csv')
slp_5min_bw = process_file('processed_data/slp_5min_bw.csv')
srp_5min_bw = process_file('processed_data/srp_5min_bw.csv')
slp5m_bw_fft = process_fft_file('processed_data/slp_5min_bw_fft.csv')
srp5m_bw_fft = process_fft_file('processed_data/srp_5min_bw_fft.csv')


In [17]:
dlp5m_bw_fft


,x,y
0,0.000000,9.383961
1,0.016668,9.384065
2,0.033336,9.384375
3,0.050004,9.384893
4,0.066671,9.385621
...,...,...
13995,233.266662,9.386563
13996,233.283330,9.385621
13997,233.299998,9.384893
13998,233.316665,9.384375


# Butterworth Filter

In [15]:
# Plotting Butterworth filtered data. Each line represents an axis.
fig, ax = plt.subplots(3)

# Diego
ax[0].plot(drp_5min_bw['seconds'], drp_5min_bw['x'])
ax[0].plot(drp_5min_bw['seconds'], drp_5min_bw['y'])
ax[0].plot(drp_5min_bw['seconds'], drp_5min_bw['z'])

# Matt
ax[1].plot(mrp_5min_bw['seconds'], mrp_5min_bw['x'])
ax[1].plot(mrp_5min_bw['seconds'], mrp_5min_bw['y'])
ax[1].plot(mrp_5min_bw['seconds'], mrp_5min_bw['z'])

# Sam
ax[2].plot(slp_5min_bw['seconds'], slp_5min_bw['x'])
ax[2].plot(slp_5min_bw['seconds'], slp_5min_bw['y'])
ax[2].plot(slp_5min_bw['seconds'], slp_5min_bw['z'])


# Fourier Transform

(0.0, 1.0)

In [16]:
# Plotting Fourier transformed data. Each line represents a collection point (left pocket vs. right pocket).
fig, ax = plt.subplots(3)

# Diego 5min
ax[0].plot(dlp5m_bw_fft['x'], dlp5m_bw_fft['y'])
ax[0].plot(drp5m_bw_fft['x'], drp5m_bw_fft['y'])
ax[0].set_xlim(0, 1)

# Matt 5min
ax[1].plot(mrp5m_bw_fft['x'], mrp5m_bw_fft['y'])
ax[1].set_xlim(0, 1)

# Sidak 5min
ax[2].plot(slp5m_bw_fft['x'], slp5m_bw_fft['y'])
ax[2].plot(srp5m_bw_fft['x'], srp5m_bw_fft['y'])
ax[2].set_xlim(0, 1)
